In [1]:
import os
import sys
import imgaug as ia
import numpy as np
import scipy
import math
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from imgaug import augmenters as iaa
from scipy import misc
from glob import glob
from tempfile import mkstemp

in_dir = "/Users/Alex/Desktop/data_for_frcnn"

gt_file = "%s/ground_truth_imgs_info.txt" % in_dir


# Define all the characters we have
imgs = {}
with open(gt_file, "r") as gt:
    for line in gt:
        comps = line.rstrip().split(",")
        img_id = os.path.splitext(os.path.basename(comps[0]))[0].replace("img", "")
        if img_id not in imgs:
            imgs[img_id] = []
        imgs[img_id].append({
            "x1": int(comps[1]),
            "y1": int(comps[2]),
            "x2": int(comps[3]),
            "y2": int(comps[4]),
            "char": comps[5]
        })

In [21]:
def augment_imgs(data):
    def affine():
        print("- Invoking affine")
        # Affine transformation
        TRANSLATE_PCT_RANGE = 0.5
        ROTATION_RANGE = (-45,45)
        SHEAR_RANGE = (-5,5)

        translate_percent = {
            "x": (-TRANSLATE_PCT_RANGE, +TRANSLATE_PCT_RANGE),
            "y": (-TRANSLATE_PCT_RANGE, +TRANSLATE_PCT_RANGE),
        }
        rotate=ROTATION_RANGE
        shear=SHEAR_RANGE
        mode = "edge"

        return iaa.Affine(translate_percent=translate_percent,
                          rotate=rotate,
                          shear=shear,
                          mode=mode)

    def add_neg():
        print("- Invoking add_neg")
        # Applies a negative to all channels
        return iaa.Add((-45, 0))

    def add_pos():
        print("- Invoking add_pos")
        # Applies a positive to all channels
        return iaa.Add((0, 45))

    def mul_neg():
        print("- Invoking mul_neg")
        # Multiples all channels by a negative factor
        return iaa.Multiply((-2, 1))

    def mul_pos():
        print("- Invoking mul_pos")
        # Multiples all channels by a postive factor
        return iaa.Multiply((1, 2))

    def blur():
        print("- Invoking blur")
        # Chooses one of three blur methods
        return one_of([
            iaa.GaussianBlur((0, 3.0)),
            iaa.AverageBlur(k=(2, 4)),
            iaa.MedianBlur(k=(3, 5)),
        ]) 
    
    def sharpen():
        print("- Invoking sharpen")
        return iaa.Sharpen(alpha=(0.5,1), lightness=(0.75,1.5))

    def invert():
        print("- Invoking invert")
        return iaa.Invert(p=1)
    
    def add_to_hue_and_sat():
        print("- Invoking add_to_hue_and_sat")
        return iaa.AddToHueAndSaturation((-20, 20))

    def scale():
        print("- Invoking scale")
        return iaa.Affine(scale = {"x": (0.8,1.2), "y": (0.8,1.2)}, mode = "edge")
    
    def piecewise_affine():
        print("- Invoking piecewise_affine")
        return iaa.PiecewiseAffine((0.0010, 0.0045))
    
    # Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
    # e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
    def sometimes(aug, pct = 0.5):
        return iaa.Sometimes(pct, aug)

    def one_of(funcs):
        # Shortcut for iaa.OneOf
        return iaa.OneOf(funcs)

    def valid_keypoints(kpts, img_ptr):
        # Returns any keypoints that are outside the width/height of the image
        width = img_ptr.shape[1]
        height = img_ptr.shape[0]
        # Copy over the "valid" keypoints (assume all are valid)
        for k in kpts:
            # If hidden, remove this runner
            if k.x < 0 or k.x > width or k .y < 0 or k.y > height:
                # Remove from valid if hidden
                valid_keypoints = [k for k in valid_keypoints if k is not kpts]
                break
        # Whatever remains becomes the Bib click points for these runners
        return valid_keypoints
    

    # Extract every char and turn it into an ia.Keypoint
    keypoints_on_images = []
    for img_id, img_data in data.items():
        img_chars = img_data["chars"]
        img_ptr = img_data["img_ptr"]
        kpts_pt1 = [ia.Keypoint(x=char["x1"], y=char["y1"]) for char in img_chars]
        kpts_pt2 = [ia.Keypoint(x=char["x2"], y=char["y2"]) for char in img_chars]
        keypoints_on_images.append(ia.KeypointsOnImage(kpts_pt1, shape=img_ptr.shape))
        keypoints_on_images.append(ia.KeypointsOnImage(kpts_pt2, shape=img_ptr.shape))
    
    seq = iaa.Sequential(
        [
            affine(),
            sometimes(one_of([add_pos(), add_neg()])),
            sometimes(one_of([mul_pos(), mul_neg()])),
            sometimes(blur(), 0.3),
            sometimes(invert()),
            sometimes(add_to_hue_and_sat()),
            sometimes(scale(), 0.60),
            sometimes(piecewise_affine())
        ],
        random_order=True
    )
    
    img_ids = [img_id for img_id, _ in data.items()]
    seq_det = seq.to_deterministic()
    try:
        aug_imgs = dict(zip(img_ids, 
                         seq_det.augment_images([v["img_ptr"] for k, v in data.items()])
                        ))
        aug_kpts = seq_det.augment_keypoints(keypoints_on_images)
    except AssertionError:
        return None
    
    from itertools import izip
    def pairwise(iterable):
        a = iter(iterable)
        return izip(a, a)
    
    aug_chars = {}
    for img_id, aug_char in dict(zip(img_ids, aug_kpts)).items():
        width, height, _ = aug_imgs[img_id].shape
        if img_id not in aug_chars:
            aug_chars[img_id] = []
        for aug_kpts in pairwise(aug_kpts):
            aug_kpt1, aug_kpt2 = aug_kpts
            total_kpts = len(aug_kpt1.keypoints)
            for i in range(total_kpts):
                char = data[img_id]["chars"][i]["char"]
                kpt1 = aug_kpt1.keypoints[i]
                kpt2 = aug_kpt2.keypoints[i]
                # Need to sort these such that {x,y}1 is min and
                # that {x,y}2 is max
                min_x = min(kpt1.x, kpt2.x)
                min_y = min(kpt1.y, kpt2.y)
                max_x = max(kpt1.x, kpt2.x)
                max_y = max(kpt1.y, kpt2.y)
                # Remove any invalid chars (points off screen)
                if min_x < 0 or max_x > width or min_y < 0 or max_y > height:
                    continue
                aug_chars[img_id].append({
                    "x1": min_x,
                    "y1": min_y,
                    "x2": max_x,
                    "y2": max_y,
                    "char": char
                })
    return aug_imgs, aug_chars

def save_image(out_dir, batch_name, img_ptr, img_id, chars, augment_no = None):
    unique_id = "%s_%s" % (img_id, "org" if augment_no is None else ("aug%s" % augment_no))
    img_path = "%s/%s.jpg" % (out_dir, unique_id)
    misc.imsave(img_path, img_ptr)
    with open("%s/%s.csv" % (out_dir, batch_name), "a+") as csv:
        for char in chars:
            csv.write(",".join([
                img_path,
                str(char["x1"]),
                str(char["y1"]),
                str(char["x2"]),
                str(char["y2"]),
                char["char"],
            ]))
            csv.write("\n")

def augment_batch(imgs, out_dir, batch_name, num_times = 50):
    assert batch_name in ["imgs", "imgs_bw"]
    img_prefix = {
        "imgs": "img",
        "imgs_bw": "img_bw"
    }[batch_name]
    gt_data = {}
    for img_id, chars in imgs.items():
        print("Loading %s in batch %s..." % (img_id, batch_name))
        img_path = "%s/%s/%s%s.jpg" % (in_dir, batch_name, img_prefix, img_id)
        img_ptr = misc.imread(img_path)
        gt_data[img_id] = {
            "img_ptr": img_ptr,
            "chars": chars
        }
        # Firstly, save the original image
        save_image(out_dir, batch_name, img_ptr, img_id, chars)
    # Batch augment data num_times
    for i in range(num_times):
        print("Augmentation Round %i/%i..." % (i + 1, num_times))
        augment_data = augment_imgs(gt_data)
        if augment_data is None:
            print("Augmentation failure")
            continue
        augmented_img_ptrs, augmented_chars = augment_data
        for img_id, img_ptr in augmented_img_ptrs.items():
            augmented_chars_for_img = augmented_chars[img_id]
            if len(augmented_chars_for_img) > 0:
                save_image(out_dir, batch_name, img_ptr, img_id, augmented_chars_for_img, i)
            
augment_batch(imgs, "/Users/Alex/Desktop/foobar", "imgs")

Loading 0001 in batch imgs...
Augmentation Round 1/50...
- Invoking affine
- Invoking add_pos
- Invoking add_neg
- Invoking mul_pos
- Invoking mul_neg
- Invoking blur
- Invoking invert
- Invoking add_to_hue_and_sat
- Invoking scale
- Invoking piecewise_affine
Augmentation Round 2/50...
- Invoking affine
- Invoking add_pos
- Invoking add_neg
- Invoking mul_pos
- Invoking mul_neg
- Invoking blur
- Invoking invert
- Invoking add_to_hue_and_sat
- Invoking scale
- Invoking piecewise_affine
Augmentation Round 3/50...
- Invoking affine
- Invoking add_pos
- Invoking add_neg
- Invoking mul_pos
- Invoking mul_neg
- Invoking blur
- Invoking invert
- Invoking add_to_hue_and_sat
- Invoking scale
- Invoking piecewise_affine
Augmentation Round 4/50...
- Invoking affine
- Invoking add_pos
- Invoking add_neg
- Invoking mul_pos
- Invoking mul_neg
- Invoking blur
- Invoking invert
- Invoking add_to_hue_and_sat
- Invoking scale
- Invoking piecewise_affine
Augmentation failure
Augmentation Round 5/50...
- 